In [42]:
import dataclasses
from typing import Dict, List, Optional

# TERMINAL_FILE = "day7_example.txt"
TERMINAL_FILE = "day7_input1.txt"

@dataclasses.dataclass
class DeviceFile():
    name: str
    size: int
    
@dataclasses.dataclass
class DeviceDirectory():
    name: str
    parent: Optional['DeviceDirectory'] = None
    children: Dict[str, 'DeviceDirectory'] = dataclasses.field(default_factory=dict) 
    files: List[DeviceFile] = dataclasses.field(default_factory=list)

    @property
    def pwd(self):
        if self.parent is None:
            return "/"
        return self.parent.pwd + self.name + "/"

    @property
    def size(self):
        file_size = sum([f.size for f in self.files])
        dir_size = sum([self.children[d].size for d in self.children.keys()])
        return file_size + dir_size

with open(TERMINAL_FILE, "r") as f:
    terminal_lines = [line.strip() for line in f.readlines()]

root_dir = DeviceDirectory(name="/")
cur_dir: Optional[DeviceDirectory] = None
cur_command = None

for line in terminal_lines:
    line_parts = line.split(" ")
    if line_parts[0] == "$": # executed command
        cur_command = line_parts[1]
        if cur_command == "cd": # change directory
            if line_parts[2] == "/": # root directory
                cur_dir = root_dir
            elif line_parts[2] == "..": # parent directory
                cur_dir = cur_dir.parent
            else: # subdirectory
                cur_dir = cur_dir.children[line_parts[2]]
        elif cur_command == "ls": # list
            continue # no action needed, parse next lines for contents
    else: # command result
        if line_parts[0] == "dir": # directory
            cur_dir.children[line_parts[1]] = DeviceDirectory(name=line_parts[1], parent=cur_dir)
        else: # file
            cur_dir.files.append(DeviceFile(name=line_parts[1], size=int(line_parts[0])))

# print(root_dir)
print(f"Total size of filesystem: {root_dir.size}")

# Find total size of all directories with max size 100000
def get_size_with_max(dir: DeviceDirectory, total: int, max_size: int) -> int:
    if dir.size <= max_size:
        total += dir.size
    for child in dir.children.values():
        total = get_size_with_max(child, total, max_size)
    return total

MAX_SIZE = 100000
total_size_with_max = get_size_with_max(root_dir, 0, MAX_SIZE)
print(f"Total recursive file size (with max={MAX_SIZE}): {total_size_with_max}")

Total size of filesystem: 43837783
Total recursive file size (with max=100000): 1454188


### Day 7, Part 2

In [43]:
TOTAL_SPACE = 70000000
UNUSED_SPACE_NEEDED = 30000000

# Find smallest directory to delete
def find_directory_to_delete(cur_dir: DeviceDirectory, del_dir: DeviceDirectory, space_needed: int):
    if cur_dir.size < del_dir.size and cur_dir.size >= space_needed:
        del_dir = cur_dir
    for child in cur_dir.children.values():
        del_dir = find_directory_to_delete(child, del_dir, space_needed)
    return del_dir

free_space = TOTAL_SPACE - root_dir.size
print(f"Free space: {free_space}")

space_needed = UNUSED_SPACE_NEEDED - free_space
print(f"Space needed: {space_needed}")

dir_to_delete = find_directory_to_delete(root_dir, root_dir, space_needed)
print(f"Recommend deleting directory `{dir_to_delete.pwd}` with size: {dir_to_delete.size}")


Free space: 26162217
Space needed: 3837783
Recommend deleting directory `/cmvqf/gccnrw/wvq/` with size: 4183246
